#Kết nối drive và import các thư viện cần thiết

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Lấy data

In [3]:
SIZE = 256

In [4]:
def parse_data(X_path, y_path):
  arr = []
  for i in os.listdir(X_path):
    arr.append(i[:-4])
  list_X = []
  list_y = []
  for i in arr:
    list_X.append(X_path+'/'+i+'.jpg')
    list_y.append(y_path+'/'+i+'.png')
  return list_X, list_y

In [7]:
## Thay đổi các đường dẫn đề phù hợp với thiết bị
images_train_path1, masks_train_path1 = parse_data('/content/drive/MyDrive/ProjectCS406/dataset/train/photos','/content/drive/MyDrive/ProjectCS406/dataset/train/masks')
images_train_path2, masks_train_path2 = parse_data('/content/drive/MyDrive/ProjectCS406/augment_data/train/photos', '/content/drive/MyDrive/ProjectCS406/augment_data/train/masks')

images_train_path = images_train_path1 + images_train_path2
masks_train_path = masks_train_path1 + masks_train_path2

In [13]:
## Thay đổi các đường dẫn đề phù hợp với thiết bị
images_val_path1, masks_val_path1 = parse_data('/content/drive/MyDrive/ProjectCS406/dataset/val/photos','/content/drive/MyDrive/ProjectCS406/dataset/val/masks')
images_val_path2, masks_val_path2 = parse_data('/content/drive/MyDrive/ProjectCS406/augment_data/val/photos','/content/drive/MyDrive/ProjectCS406/augment_data/val/masks')

images_val_path = images_val_path1 + images_val_path2
masks_val_path = masks_val_path1 + masks_val_path2


In [ ]:
## Đọc ảnh từ danh sách path
images_train = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2GRAY) for img in images_train_path]
masks_train = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2GRAY) for img in masks_train_path]
images_val = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2GRAY) for img in images_val_path]
masks_val = [cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2GRAY) for img in masks_val_path]

In [ ]:
## covert danh sách ảnh qua dạng numpy array
images_train = np.array(images_train)
images_train = np.expand_dims(images_train, axis=3)

masks_train = np.array(masks_train)
masks_train = np.expand_dims(masks_train, axis=3)

images_val = np.array(images_val)
images_val = np.expand_dims(images_val, axis=3)

masks_val = np.array(masks_val)
masks_val = np.expand_dims(masks_val, axis=3)

In [ ]:
print("Image data shape is: ", images_val.shape)
print("Mask data shape is:", masks_val.shape)
print("Max pixel value in image is: ", images_val.max())
print("Labels in the mask are: ", np.unique(masks_val))

# Normalizations iamges

In [ ]:
# Normalize images
images_train = images_train/255

masks_train = masks_train/255

images_val = images_val/255

masks_val = masks_val/255


#Build model 

In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Dropout, MaxPooling2D, Input, BatchNormalization, Lambda, Activation, concatenate, Conv2DTranspose, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding="same")(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv2D(num_filters, 3, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

def encoder_block(input, num_filters):
  x = conv_block(input, num_filters)
  p = MaxPool2D((2,2))(x)
  return x, p

def decoder_block(input, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(input)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

def build_unet(input_shape, n_classes):
  inputs = Input(input_shape)

  s1,p1 = encoder_block(inputs, 64)
  s2,p2 = encoder_block(p1, 128)
  s3,p3 = encoder_block(p2, 256)
  s4,p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  if n_classes == 1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'

  outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)
  print(activation)
  
  model = Model(inputs, outputs, name="U-Net")
  return model



## Setup các parameter

In [ ]:
IMAGE_HEIGHT = images_train.shape[1]
IMAGE_WIDTH = images_train.shape[2]
IMAGE_CHANNELS = images_train.shape[3]

input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

In [ ]:
model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## Tiến hành training

In [ ]:
checkpoint_path = "/content/drive/MyDrive/ProjectCS406/New/checkpoint0/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              period=25,
                                              verbose=1)

history = model.fit(images_train, masks_train,
                    batch_size=16, 
                    verbose=1,
                    epochs=100,
                    callbacks=[cp_callback],
                    validation_data=(images_val, masks_val),
                    shuffle=False)

Epoch 1/100
53/53 [==============================] - 73s 1s/step - loss: 0.4131 - accuracy: 0.8188 - val_loss: 93.4089 - val_accuracy: 0.6628
Epoch 2/100
53/53 [==============================] - 45s 851ms/step - loss: 0.3296 - accuracy: 0.8614 - val_loss: 0.9784 - val_accuracy: 0.6477
Epoch 3/100
53/53 [==============================] - 46s 869ms/step - loss: 0.2975 - accuracy: 0.8770 - val_loss: 4.0522 - val_accuracy: 0.6630
Epoch 4/100
53/53 [==============================] - 46s 875ms/step - loss: 0.2713 - accuracy: 0.8871 - val_loss: 5.2298 - val_accuracy: 0.6628
Epoch 5/100
53/53 [==============================] - 47s 892ms/step - loss: 0.2635 - accuracy: 0.8895 - val_loss: 1.6149 - val_accuracy: 0.6682
Epoch 6/100
53/53 [==============================] - 47s 895ms/step - loss: 0.2552 - accuracy: 0.8933 - val_loss: 0.7609 - val_accuracy: 0.7416
Epoch 7/100
53/53 [==============================] - 48s 903ms/step - loss: 0.2432 - accuracy: 0.8986 - val_loss: 0.5710 - val_accuracy: 0

# lưu model

In [ ]:
model.save('/content/drive/MyDrive/ProjectCS406/New/model_unet_aug.hdf5')